In [1]:
import sys
import os
sys.path.append('.')

import numpy as np
import pandas as pd
import pickle as pkl
from os.path import exists
from data_loader.loader import load_tables, DataLoader

Using TensorFlow backend.


In [2]:
tables = load_tables('prior')

In [3]:
np.unique(tables['orders']['order_hour_of_day'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [4]:
order_items = tables['prior'].groupby('order_id')['product_id'].apply(lambda x : np.array(x))

KeyboardInterrupt: 

In [5]:
order_items

order_id
2          [33120, 28985, 9327, 45918, 30035, 17794, 4014...
3          [33754, 24838, 17704, 21903, 17668, 46667, 174...
4          [46842, 26434, 39758, 27761, 10054, 21351, 225...
5          [13176, 15005, 47329, 27966, 23909, 48370, 132...
6                                      [40462, 15873, 41897]
                                 ...                        
3421079                                              [30136]
3421080    [27845, 4932, 18811, 41950, 31717, 12935, 2512...
3421081     [38185, 12218, 32299, 3060, 20539, 35221, 12861]
3421082    [17279, 12738, 16797, 43352, 32700, 12023, 47941]
3421083    [7854, 45309, 21162, 18176, 35211, 39678, 1135...
Name: product_id, Length: 3214874, dtype: object

In [6]:
order_items.iloc[0]

array([33120, 28985,  9327, 45918, 30035, 17794, 40141,  1819, 43668])

In [7]:
origin_data = tables['orders']
orders0 = pd.DataFrame({'last_order_id': origin_data['order_id'],
                        'user_id': origin_data['user_id'],
                        'order_number': origin_data['order_number'] + 1})

In [8]:
orders0

,last_order_id,user_id,order_number
0,2539329,1,2
1,2398795,1,3
2,473747,1,4
3,2254736,1,5
4,431534,1,6
...,...,...,...
3421077,2558525,206209,10
3421078,2266710,206209,11
3421079,1854736,206209,12
3421080,626363,206209,13


In [9]:
origin_data2 = origin_data.merge(orders0,\
             left_on=['order_number', 'user_id'], right_on=['order_number', 'user_id'], how='left')

In [10]:
origin_data2

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_id
0,2539329,1,1,2,8,NaN,NaN
1,2398795,1,2,3,7,15.0,2539329.0
2,473747,1,3,3,12,21.0,2398795.0
3,2254736,1,4,4,7,29.0,473747.0
4,431534,1,5,4,15,28.0,2254736.0
...,...,...,...,...,...,...,...
3214869,2558525,206209,9,4,15,22.0,2129269.0
3214870,2266710,206209,10,5,18,29.0,2558525.0
3214871,1854736,206209,11,4,10,30.0,2266710.0
3214872,626363,206209,12,1,12,18.0,1854736.0


In [11]:
print(len(origin_data2.dropna()))
print(len(origin_data.dropna()))

3008665
3008665


In [12]:
input_data = origin_data2.dropna().set_index('last_order_id').\
            join(order_items).rename(columns={'product_id': 'last_order_list'})

In [13]:
input_data

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_list
last_order_id,,,,,,,
2539329.0,2398795,1,2,3,7,15.0,"[196, 14084, 12427, 26088, 26405]"
2398795.0,473747,1,3,3,12,21.0,"[196, 10258, 12427, 13176, 26088, 13032]"
473747.0,2254736,1,4,4,7,29.0,"[196, 12427, 10258, 25133, 30450]"
2254736.0,431534,1,5,4,15,28.0,"[196, 12427, 10258, 25133, 26405]"
431534.0,3367565,1,6,2,7,19.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787..."
...,...,...,...,...,...,...,...
2129269.0,2558525,206209,9,4,15,22.0,"[38167, 43961, 19348, 13176, 6846, 17500, 3605..."
2558525.0,2266710,206209,10,5,18,29.0,"[38167, 23892, 6846]"
2266710.0,1854736,206209,11,4,10,30.0,"[9405, 6846, 15700, 26503, 41213, 23800, 24852..."


In [14]:
input_data2 = input_data.set_index('order_id').join(order_items).rename(columns={'product_id': 'order_list'})

In [15]:
input_data2

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_list,order_list
order_id,,,,,,,
2398795,1,2,3,7,15.0,"[196, 14084, 12427, 26088, 26405]","[196, 10258, 12427, 13176, 26088, 13032]"
473747,1,3,3,12,21.0,"[196, 10258, 12427, 13176, 26088, 13032]","[196, 12427, 10258, 25133, 30450]"
2254736,1,4,4,7,29.0,"[196, 12427, 10258, 25133, 30450]","[196, 12427, 10258, 25133, 26405]"
431534,1,5,4,15,28.0,"[196, 12427, 10258, 25133, 26405]","[196, 12427, 10258, 25133, 10326, 17122, 41787..."
3367565,1,6,2,7,19.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787...","[196, 12427, 10258, 25133]"
...,...,...,...,...,...,...,...
2558525,206209,9,4,15,22.0,"[38167, 43961, 19348, 13176, 6846, 17500, 3605...","[38167, 23892, 6846]"
2266710,206209,10,5,18,29.0,"[38167, 23892, 6846]","[9405, 6846, 15700, 26503, 41213, 23800, 24852..."
1854736,206209,11,4,10,30.0,"[9405, 6846, 15700, 26503, 41213, 23800, 24852...","[6846, 9405, 41213, 24852, 19348, 15700, 40310..."


In [21]:
new_col = input_data2['last_order_list'].map(lambda x : len(x))

In [42]:
new_col2 = input_data['last_order_list'].map(lambda x : len(x))
new_col2.max()

145

In [35]:
def transform(order_list):
    y = np.zeros(49688)
    y[order_list] = 1
    return y

slices = input_data2[:1000]
slices['y'] = slices['order_list'].map(transform)

In [37]:
slices.drop('order_list', axis=1)

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_list,y
order_id,,,,,,,
2398795,1,2,3,7,15.0,"[196, 14084, 12427, 26088, 26405]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
473747,1,3,3,12,21.0,"[196, 10258, 12427, 13176, 26088, 13032]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2254736,1,4,4,7,29.0,"[196, 12427, 10258, 25133, 30450]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
431534,1,5,4,15,28.0,"[196, 12427, 10258, 25133, 26405]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3367565,1,6,2,7,19.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
1338309,75,20,6,11,14.0,"[22451, 21267, 48364, 27105, 12872, 27104, 100...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3192011,75,21,0,13,22.0,"[4562, 22451, 13176, 12872, 27105, 44683, 2710...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1578024,75,22,2,13,9.0,"[4562, 47626, 11520, 27105, 22451, 34969, 3098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [43]:
def transform(order_list):
    hist = np.zeros(145)
    hist[:len(order_list)] = order_list
    return hist

hist_seq = slices['last_order_list'].map(transform)

In [44]:
hist_seq

order_id
2398795    [196.0, 14084.0, 12427.0, 26088.0, 26405.0, 0....
473747     [196.0, 10258.0, 12427.0, 13176.0, 26088.0, 13...
2254736    [196.0, 12427.0, 10258.0, 25133.0, 30450.0, 0....
431534     [196.0, 12427.0, 10258.0, 25133.0, 26405.0, 0....
3367565    [196.0, 12427.0, 10258.0, 25133.0, 10326.0, 17...
                                 ...                        
1338309    [22451.0, 21267.0, 48364.0, 27105.0, 12872.0, ...
3192011    [4562.0, 22451.0, 13176.0, 12872.0, 27105.0, 4...
1578024    [4562.0, 47626.0, 11520.0, 27105.0, 22451.0, 3...
1657089    [24964.0, 21267.0, 29418.0, 19303.0, 48364.0, ...
1522904    [15460.0, 49495.0, 246.0, 36107.0, 0.0, 0.0, 0...
Name: last_order_list, Length: 1000, dtype: object

In [53]:
np.array(hist_seq.to_list())

array([[  196., 14084., 12427., ...,     0.,     0.,     0.],
       [  196., 10258., 12427., ...,     0.,     0.,     0.],
       [  196., 12427., 10258., ...,     0.,     0.,     0.],
       ...,
       [ 4562., 47626., 11520., ...,     0.,     0.,     0.],
       [24964., 21267., 29418., ...,     0.,     0.,     0.],
       [15460., 49495.,   246., ...,     0.,     0.,     0.]])

In [136]:
products = tables['prior'][['product_id']]

In [137]:
products['cnt'] = 1

In [139]:
prob = products.groupby('product_id')['cnt'].sum().map(lambda x: np.power(x, 0.75))

In [157]:
prob_dict = dict(prob / prob.sum())
print(prob_dict)

607982256e-06, 49029: 2.606731689277665e-06, 49030: 5.4194284115841764e-05, 49031: 9.941225573965557e-06, 49032: 2.939878746618415e-05, 49033: 1.249164928113151e-06, 49034: 2.1877772453005794e-06, 49035: 1.0916267293877489e-05, 49036: 5.182643525580035e-06, 49037: 1.5878934099252913e-05, 49038: 1.529548272108664e-05, 49039: 7.715925560036382e-06, 49040: 4.452305541104233e-06, 49041: 1.1435506116660288e-06, 49042: 1.0345736912396173e-06, 49043: 5.479978332604621e-07, 49044: 0.00014365081625174027, 49045: 2.606731689277665e-06, 49046: 1.8323411907050996e-06, 49047: 3.6793880858938075e-06, 49048: 2.358318030120461e-06, 49049: 6.907166172708314e-06, 49050: 3.309964646951321e-06, 49051: 1.40126162847362e-05, 49052: 6.490851367575305e-06, 49053: 1.0560553326436374e-05, 49054: 9.671753899997615e-05, 49055: 1.452052818548035e-06, 49056: 1.0345736912396173e-06, 49057: 2.358318030120461e-06, 49058: 4.921289578467126e-06, 49059: 9.216188271110508e-07, 49060: 3.874249586046086e-05, 49061: 7.487855

In [158]:
probs = np.vectorize(prob_dict.get)(np.arange(0, 49689)).astype('float32')
np.arange(0, 49689)[np.isnan(probs)]

array([    0,  3630,  3718,  7045, 25383, 27499, 36233, 37703, 43725,
       45971, 46625, 49540])

In [110]:
probs[np.isnan(probs)] = 0

In [114]:
np.random.choice(np.arange(0, 49689), p=probs)

45204

In [174]:
class NegativeSampling:
    def __init__(self, item_count, consts, sample_ratio):
        self.item_count = item_count
        prob_dict = consts['product_prob']
        self.probs = np.vectorize(prob_dict.get)(np.arange(0, item_count)).astype('float32')
        self.probs[np.isnan(self.probs)] = 0
        self.sample_ratio = sample_ratio

    def __call__(self, order_list):
        sub_samples_list = []
        pos_set = set(order_list)
        neg_list = np.array([self.sample_neg_once(pos_set) for i in range(self.sample_ratio * len(order_list))])
        for i in range(len(order_list)):
            index = np.random.randint(len(neg_list), size=self.sample_ratio)
            sub_samples_list.append(np.append(neg_list[index], order_list[i]))
        return sub_samples_list

    def sample_neg_once(self, pos):
        neg = np.random.choice(np.arange(0, self.item_count), p=self.probs)
        while neg in pos:
            neg = np.random.choice(np.arange(0, self.item_count), p=self.probs)
        return neg

In [122]:
np.random.randint(20, size=(10))

array([ 9,  2, 15,  5, 12,  3, 14, 11,  9, 15])

In [175]:
sub_sampled = slices['order_list'].map(NegativeSampling(49689, {'product_prob':prob_dict}, 20))

In [177]:
slices['sub_sampled'] = sub_sampled

In [197]:
len(slices.iloc[3,-1])

8

In [178]:
slices2 = slices.explode('sub_sampled')

In [200]:
slices2['order_list'].map(str)

order_id
2398795                [  196 10258 12427 13176 26088 13032]
2398795                [  196 10258 12427 13176 26088 13032]
2398795                [  196 10258 12427 13176 26088 13032]
2398795                [  196 10258 12427 13176 26088 13032]
2398795                [  196 10258 12427 13176 26088 13032]
                                 ...                        
1657089    [29071 21267 21903    45 26369 42028    79  84...
1657089    [29071 21267 21903    45 26369 42028    79  84...
1657089    [29071 21267 21903    45 26369 42028    79  84...
1657089    [29071 21267 21903    45 26369 42028    79  84...
1522904                                              [36107]
Name: order_list, Length: 9139, dtype: object

In [52]:
shard = pd.read_csv('data_loader/data/prior_input_shard_2.csv')
index = shard['sub_samples'].map(lambda x: not isinstance(x, str))
shard.loc[index]

,Unnamed: 0,Unnamed: 0.1,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_list,order_list,sub_samples


In [16]:
raw = pd.read_csv('data_loader/data/prior_raw.csv')

In [26]:
(raw['last_order_list'].map(type) == type('')).all()

True

In [3]:
def load_array(series, dtype='float32'):
    # transform from str type to numpy array
    def aux(mylist):
        if mylist[0] == '[':
            mylist = mylist[1:]
        if mylist[-1] == ']':
            mylist = mylist[:-1]
        if '[' == mylist[0][0]:
            mylist[0] = mylist[0][1:]
        if ']' == mylist[-1][-1]:
            mylist[-1] = mylist[-1][:-1]
        return np.array(mylist, dtype=dtype)
    return series.map(str.split).map(aux)

def load_csv_with_arrays(path, columns):
    raw = pd.read_csv(path)
    for cname, dtype in columns:
        raw[cname] = load_array(raw[cname], dtype)
    return raw

In [6]:
array_columns = [('order_list', 'int32'), ('last_order_list', 'int32'), ('sub_samples', 'int32')]
res = load_csv_with_arrays('data_loader/data/prior_input_shard_2.csv', array_columns)

In [10]:
res.size * 100 / 128 / 2

1306769.921875

In [86]:
f = open('consts.pkl', 'rb')
consts = pkl.load(f)
f.close()

In [69]:
import tensorflow as tf

In [76]:
aisle_list = tf.convert_to_tensor(consts['aisle_list'], dtype=tf.int32)
index = tf.Variable([1,2,3,4])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run(tf.gather(aisle_list, index))
    print(out)

[104  94  38   5]


In [90]:
aisle_list = consts['aisle_list']
aisle_list.insert(0, 0)
print(aisle_list[:10])

[0, 0, 0, 0, 61, 104, 94, 38, 5, 11]


In [2]:
tables = load_tables('train')

In [8]:
orders = tables['orders']
train_orders = orders[orders['eval_set'] == 'train']

In [4]:
len(orders)

3346083

In [21]:
prior_order_items = tables['prior'].groupby('order_id')['product_id'].apply(lambda x : np.array(x))

In [22]:
train_order_items = tables['train'].groupby('order_id')['product_id'].apply(lambda x : np.array(x))

In [17]:
orders00 = orders.sort_values(['user_id', 'order_number'], ascending=False).groupby('user_id', as_index=False).head(2)
orders0 = pd.DataFrame({'last_order_id': orders00['order_id'],
                        'user_id': orders00['user_id'],
                        'order_number': orders00['order_number'] + 1})
orders0

,last_order_id,user_id,order_number
3421082,272231,206209,15
3421081,2977660,206209,14
3421067,1882108,206208,50
3421066,1285346,206208,49
3421017,1005822,206207,17
...,...,...,...
36,521107,3,12
25,1492625,2,16
24,839880,2,15
10,1187899,1,12


In [18]:
origin_data = train_orders.merge(orders0, on=['user_id', 'order_number'], how='left')

In [19]:
origin_data

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order_id
0,1187899,1,train,11,4,8,14.0,2550362
1,1492625,2,train,15,1,11,30.0,839880
2,2196797,5,train,5,0,11,6.0,157374
3,525192,7,train,21,2,11,6.0,2452257
4,880375,8,train,4,1,14,10.0,2570360
...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,1932632
131205,943915,206200,train,24,6,19,6.0,1909878
131206,2371631,206203,train,6,4,19,30.0,133582
131207,1716008,206205,train,4,1,16,10.0,414137


In [20]:
origin_data.merge('')

131209